In [1]:
import dolfin as dl
import matplotlib.pyplot as plt
import numpy as np

from spin.core import problem

In [11]:
#mesh = dl.IntervalMesh(100, -1, 1)
mesh = dl.RectangleMesh(dl.Point(-1, -1), dl.Point(1, 1), 10, 10)
settings = problem.SPINProblemSettings(
    mesh=mesh,
    pde_type="mean_exit_time",
    inference_type="drift_only",
    drift = (("-x[0]", "-x[1]")),
    log_squared_diffusion = ("std::log(1.0)", "std::log(1.0)"),
    element_degree_variables=2,
)

In [12]:
builder = problem.SPINProblemBuilder(settings)
variational_problem = builder.build()

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [13]:
forward_vector = variational_problem.generate_state()
adjoint_vector = variational_problem.generate_state()
parameter_vector = variational_problem.generate_parameter()
parameter_vector.set_local(-np.linspace(-1, 1, parameter_vector.size()))
state_list = [forward_vector, parameter_vector, adjoint_vector]
variational_problem.solveFwd(forward_vector, state_list)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [14]:
var, drift, diff = builder.coordinates

In [16]:
forward_vector.get_local().shape

(441,)

In [15]:
var.shape

(441, 2)